This notebook demonstrates how to concatenate two CSV files, standardize column names, and ingest the resulting data into a SQL Server database running on Docker. We will be using Pandas for data manipulation and pymssql for interacting with the SQL Server.

Prerequistes
Befor running this notebook, ensure that you have the following installed:
- Python
- Jupyter Notebook
- Docker
- SQL Server Docker container running with ports exposed : https://hub.docker.com/r/microsoft/mssql-server
- Necessary Python libraries:
    - pandas
    - pymssql 

In [8]:
# Import libraries and modules
import pymssql
import pandas as pd
import numpy as np
from pwd import user_name, passwrd

In [9]:
# Get all the credientials here

server = 'localhost'
database = 'master'
username = user_name 
password = passwrd

In [10]:
# Check if the connection is successful or not
try:
    conn = pymssql.connect(server=server,  user=username, password=password, database=database,autocommit=True)
    print("connection successful")
except Exception as e:
    print(f"Error : {e}")

connection successful


In [11]:
# Now the connection is successful, create a sql cursor to interact with the database
cursor = conn.cursor()
cursor.execute("""
               IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = 'crimedatabase')
               BEGIN
                    CREATE DATABASE crimedatabase;
               END""")

In [12]:
# Get the data from CSV files into pandas dataframe
data_10_19 = pd.read_csv('C:/Users/Varun/Downloads/Crime_Data_from_2010_to_2019.csv')
data_20_24 = pd.read_csv('C:/Users/Varun/Downloads/Crime_Data_from_2020_to_present.csv')

In [13]:
# quick look
print(f'LA Crime data 2010 - 2019: \n{data_10_19.head()}')
print(f'\n LA Crime data 2020 - 2024: \n{data_20_24.head()}')

LA Crime data 2010 - 2019: 
       DR_NO               Date Rptd                DATE OCC  TIME OCC  AREA   \
0    1307355  02/20/2010 12:00:00 AM  02/20/2010 12:00:00 AM      1350     13   
1   11401303  09/13/2010 12:00:00 AM  09/12/2010 12:00:00 AM        45     14   
2   70309629  08/09/2010 12:00:00 AM  08/09/2010 12:00:00 AM      1515     13   
3   90631215  01/05/2010 12:00:00 AM  01/05/2010 12:00:00 AM       150      6   
4  100100501  01/03/2010 12:00:00 AM  01/02/2010 12:00:00 AM      2100      1   

   AREA NAME  Rpt Dist No  Part 1-2  Crm Cd  \
0     Newton         1385         2     900   
1    Pacific         1485         2     740   
2     Newton         1324         2     946   
3  Hollywood          646         2     900   
4    Central          176         1     122   

                                         Crm Cd Desc  ... Status  \
0                           VIOLATION OF COURT ORDER  ...     AA   
1  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...  ...     IC  

In [14]:
# Quick look at the data types and descriptions
data_10_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123044 entries, 0 to 2123043
Data columns (total 28 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DR_NO           int64  
 1   Date Rptd       object 
 2   DATE OCC        object 
 3   TIME OCC        int64  
 4   AREA            int64  
 5   AREA NAME       object 
 6   Rpt Dist No     int64  
 7   Part 1-2        int64  
 8   Crm Cd          int64  
 9   Crm Cd Desc     object 
 10  Mocodes         object 
 11  Vict Age        int64  
 12  Vict Sex        object 
 13  Vict Descent    object 
 14  Premis Cd       float64
 15  Premis Desc     object 
 16  Weapon Used Cd  float64
 17  Weapon Desc     object 
 18  Status          object 
 19  Status Desc     object 
 20  Crm Cd 1        float64
 21  Crm Cd 2        float64
 22  Crm Cd 3        float64
 23  Crm Cd 4        float64
 24  LOCATION        object 
 25  Cross Street    object 
 26  LAT             float64
 27  LON             float64
dtypes: float64(8

In [15]:
# Quick look at the data types and descriptions
data_20_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982638 entries, 0 to 982637
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           982638 non-null  int64  
 1   Date Rptd       982638 non-null  object 
 2   DATE OCC        982638 non-null  object 
 3   TIME OCC        982638 non-null  int64  
 4   AREA            982638 non-null  int64  
 5   AREA NAME       982638 non-null  object 
 6   Rpt Dist No     982638 non-null  int64  
 7   Part 1-2        982638 non-null  int64  
 8   Crm Cd          982638 non-null  int64  
 9   Crm Cd Desc     982638 non-null  object 
 10  Mocodes         837376 non-null  object 
 11  Vict Age        982638 non-null  int64  
 12  Vict Sex        844193 non-null  object 
 13  Vict Descent    844182 non-null  object 
 14  Premis Cd       982624 non-null  float64
 15  Premis Desc     982053 non-null  object 
 16  Weapon Used Cd  326167 non-null  float64
 17  Weapon Des

In [16]:
# Concatenate these two files together as we know now that they have same columns and datatypes
complete_data = pd.concat([data_10_19,data_20_24],axis=0,ignore_index=True)

In [17]:
# Check the total 
complete_data.shape

(3105682, 29)

In [18]:
complete_data.info()
# We see here that Column name AREA has space issue in the name

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 29 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DR_NO           int64  
 1   Date Rptd       object 
 2   DATE OCC        object 
 3   TIME OCC        int64  
 4   AREA            float64
 5   AREA NAME       object 
 6   Rpt Dist No     int64  
 7   Part 1-2        int64  
 8   Crm Cd          int64  
 9   Crm Cd Desc     object 
 10  Mocodes         object 
 11  Vict Age        int64  
 12  Vict Sex        object 
 13  Vict Descent    object 
 14  Premis Cd       float64
 15  Premis Desc     object 
 16  Weapon Used Cd  float64
 17  Weapon Desc     object 
 18  Status          object 
 19  Status Desc     object 
 20  Crm Cd 1        float64
 21  Crm Cd 2        float64
 22  Crm Cd 3        float64
 23  Crm Cd 4        float64
 24  LOCATION        object 
 25  Cross Street    object 
 26  LAT             float64
 27  LON             float64
 28  AREA        

In [19]:
complete_data.rename(columns={'DR_NO':'file_no',
                              'Date Rptd':'date_rptd',
                              'DATE OCC':'date_occ',
                              'TIME OCC':'time_occ',
                              'AREA ':'area_code',
                              'AREA NAME':'area_name',
                              'Rpt Dist No':'rpt_dist_no',
                              'Crm Cd':'crm_code',
                              'Crm Cd Desc':'crm_code_description',
                              'Mocodes':'modus_operandi',
                              'Vict Age':'victim_age',
                              'Vict Sex':'victim_sex',
                              'Vict Descent':'victim_descent',
                              'Premis Cd':'premise_code',
                              'Premis Desc':'premise_code_desc',
                              'Weapon Used Cd':'weapon_used_code',
                              'Weapon Desc':'weapon_desc',
                              'Status':'status_code',
                              'Status Desc':'status_desc',
                              'Crm Cd 1':'crm_code_1',
                              'Crm Cd 2':'crm_code_2',
                              'Crm Cd 3':'crm_code_3',
                              'Crm Cd 4':'crm_code_4',
                              'LOCATION':'crm_location',
                              'Cross Street':'cross_street',
                              'LAT':'latitude',
                              'LON':'longitude'},inplace=True)

In [20]:
complete_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3105682 entries, 0 to 3105681
Data columns (total 29 columns):
 #   Column                Dtype  
---  ------                -----  
 0   file_no               int64  
 1   date_rptd             object 
 2   date_occ              object 
 3   time_occ              int64  
 4   area_code             float64
 5   area_name             object 
 6   rpt_dist_no           int64  
 7   Part 1-2              int64  
 8   crm_code              int64  
 9   crm_code_description  object 
 10  modus_operandi        object 
 11  victim_age            int64  
 12  victim_sex            object 
 13  victim_descent        object 
 14  premise_code          float64
 15  premise_code_desc     object 
 16  weapon_used_code      float64
 17  weapon_desc           object 
 18  status_code           object 
 19  status_desc           object 
 20  crm_code_1            float64
 21  crm_code_2            float64
 22  crm_code_3            float64
 23  crm_cod

In [21]:
# Let's fix this issue before we drop them or proceed
complete_data['area_code'] = np.where(complete_data['area_code'].isna(),complete_data['AREA'],complete_data['area_code'])

In [22]:
complete_data.drop(columns={'Part 1-2','AREA'},inplace=True)
complete_data.replace({np.nan:None},inplace=True)

In [23]:
# final quick check
print(f'top rows: \n{complete_data.head()}')
print(f'\n Quick desc: \n{complete_data.info()}')

top rows: 
     file_no               date_rptd                date_occ  time_occ  \
0    1307355  02/20/2010 12:00:00 AM  02/20/2010 12:00:00 AM      1350   
1   11401303  09/13/2010 12:00:00 AM  09/12/2010 12:00:00 AM        45   
2   70309629  08/09/2010 12:00:00 AM  08/09/2010 12:00:00 AM      1515   
3   90631215  01/05/2010 12:00:00 AM  01/05/2010 12:00:00 AM       150   
4  100100501  01/03/2010 12:00:00 AM  01/02/2010 12:00:00 AM      2100   

   area_code  area_name  rpt_dist_no  crm_code  \
0       13.0     Newton         1385       900   
1       14.0    Pacific         1485       740   
2       13.0     Newton         1324       946   
3        6.0  Hollywood          646       900   
4        1.0    Central          176       122   

                                crm_code_description  modus_operandi  ...  \
0                           VIOLATION OF COURT ORDER  0913 1814 2000  ...   
1  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...            0329  ...   
2           

In [75]:
# Now time for data deployment into the database 
cursor.execute("""
               IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'crimedata' AND TABLE_SCHEMA='dbo')

                BEGIN
                      CREATE  TABLE crimedatabase.dbo.crimedata (
                                                                                  file_no int PRIMARY KEY,
                                                                                  date_rptd date,
                                                                                  date_occ date,
                                                                                  time_occ int,
                                                                                  area_code float,
                                                                                  area_name varchar(255),
                                                                                  rpt_dist_no int,
                                                                                  crm_code int,
                                                                                  crm_code_description varchar(255),
                                                                                  modus_operandi varchar(255),
                                                                                  victim_age int,
                                                                                  victim_sex varchar(10),
                                                                                  vitim_descent varchar(100),
                                                                                  premise_code float,
                                                                                  premise_code_desc varchar(255),
                                                                                  weapon_used_code float,
                                                                                  weapon_desc varchar(255),
                                                                                  status_code varchar(100),
                                                                                  status_desc varchar(255),
                                                                                  crm_code_1 float,
                                                                                  crm_code_2 float,
                                                                                  crm_code_3 float,
                                                                                  crm_code_4 float,
                                                                                  crm_location varchar(255),
                                                                                  cross_street varchar(255),
                                                                                  latitude float,
                                                                                  longitude float                    
                                                                                  )
                END
                """)

In [76]:
row_count = 0
for index, row in complete_data.iterrows():
    row_count +=1
    if row_count%100000 == 0:
        print('Row entered: ',row_count)
    cursor.execute("""
                        INSERT INTO crimedatabase.dbo.crimedata (
                                                                    file_no,
                                                                    date_rptd,
                                                                    date_occ,
                                                                    time_occ,
                                                                    area_code,
                                                                    area_name,
                                                                    rpt_dist_no,
                                                                    crm_code,
                                                                    crm_code_description,
                                                                    modus_operandi,
                                                                    victim_age,
                                                                    victim_sex,
                                                                    vitim_descent,
                                                                    premise_code,
                                                                    premise_code_desc,
                                                                    weapon_used_code,
                                                                    weapon_desc,
                                                                    status_code,
                                                                    status_desc,
                                                                    crm_code_1,
                                                                    crm_code_2,
                                                                    crm_code_3,
                                                                    crm_code_4,
                                                                    crm_location,
                                                                    cross_street,
                                                                    latitude,
                                                                    longitude                    
                                                                )
                        VALUES (    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %d,
                                    %s,
                                    %s,
                                    %d,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s,
                                    %s)
                            """,
                            (
                                row['file_no'],row['date_rptd'],row['date_occ'],row['time_occ'],row['area_code'],row['area_name'],row['rpt_dist_no'],
                                row['crm_code'],row['crm_code_description'],row['modus_operandi'],row['victim_age'],row['victim_sex'],row['victim_descent'],
                                row['premise_code'],row['premise_code_desc'],row['weapon_used_code'],row['weapon_desc'],row['status_code'],row['status_desc'],
                                row['crm_code_1'],row['crm_code_2'],row['crm_code_3'],row['crm_code_4'],row['crm_location'],row['cross_street'],row['latitude'],row['longitude']
                            )
                        )

Row entered:  100000
Row entered:  200000
Row entered:  300000
Row entered:  400000
Row entered:  500000
Row entered:  600000
Row entered:  700000
Row entered:  800000
Row entered:  900000
Row entered:  1000000
Row entered:  1100000
Row entered:  1200000
Row entered:  1300000
Row entered:  1400000
Row entered:  1500000
Row entered:  1600000
Row entered:  1700000
Row entered:  1800000
Row entered:  1900000
Row entered:  2000000
Row entered:  2100000
Row entered:  2200000
Row entered:  2300000
Row entered:  2400000
Row entered:  2500000
Row entered:  2600000
Row entered:  2700000
Row entered:  2800000
Row entered:  2900000
Row entered:  3000000
Row entered:  3100000


In [ ]:
# I made an error in area_code column so created another table and Trigger to fix this issue...
cursor.execute("""
               IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'fixeddata' AND TABLE_SCHEMA='dbo')

                BEGIN
                      CREATE  TABLE crimedatabase.dbo.fixeddata (
                                                                                  file_no int PRIMARY KEY,
                                                                                  area_code float                    
                                                                                  )
                END
                """)

In [25]:
fixed_data = complete_data[['file_no','area_code']]

In [26]:
fixed_data.head()

,file_no,area_code
0,1307355,13.0
1,11401303,14.0
2,70309629,13.0
3,90631215,6.0
4,100100501,1.0


In [45]:
row_count = 935809
for index, row in fixed_data.iloc[935810:].iterrows():
    row_count +=1
    if row_count%100000 == 0:
        print('Row entered: ',row_count)
    cursor.execute("""
                        INSERT INTO crimedatabase.dbo.fixeddata (
                                                                    file_no,                                                                    
                                                                    area_code                   
                                                                )
                        VALUES (    %d,
                                    %s
                                )
                            """,
                            (
                                int(row['file_no']),float(row['area_code'])
                            )
                        )

Row entered:  1000000
Row entered:  1100000
Row entered:  1200000
Row entered:  1300000
Row entered:  1400000
Row entered:  1500000
Row entered:  1600000
Row entered:  1700000
Row entered:  1800000
Row entered:  1900000
Row entered:  2000000
Row entered:  2100000
Row entered:  2200000
Row entered:  2300000
Row entered:  2400000
Row entered:  2500000
Row entered:  2600000
Row entered:  2700000
Row entered:  2800000
Row entered:  2900000
Row entered:  3000000
Row entered:  3100000
